In [2]:
from math import exp

import pandas as pd
import numpy as np
from scipy.special import gamma as GAMMA

In [3]:
def get_fractional_series(X0, zeta):
    """
    计算序列的分数阶
    :param X0: 待计算分数阶的序列
    :param zeta: zeta
    :return: 分数阶序列
    """
    len_x0 = len(X0)

    # 初始化数组
    xishu = np.zeros((len_x0, len_x0))
    Xr = np.zeros(len_x0)

    for k in range(len_x0):
        tmp = 0
        for i in range(k + 1):
            xishu[k, i] = GAMMA(zeta + k - i) / (GAMMA(k - i + 1) * GAMMA(zeta))
            tmp += xishu[k, i] * X0[i]
        Xr[k] = tmp

    return Xr


def func(XX):
    zeta, tau, gamma = XX
    # 第一列是年份，故先舍弃
    X0 = data.iloc[:train_n, 1:].to_numpy()

    X1 = np.array([get_fractional_series(X0[:, 0], zeta),
                   get_fractional_series(X0[:, 1], zeta),
                   get_fractional_series(X0[:, 2], zeta)]).T

    y1 = X1[:, 0] ** (1 - gamma)

    Zy = (y1[1:] + y1[:-1]) / 2
    M2 = (X1[1:, 1] + X1[:-1, 1]) / 2
    M3 = (X1[1:, 2] + X1[:-1, 2]) / 2

    # 构建 B
    TAU = (np.exp(tau) - 1) / tau * np.exp(tau * (np.arange(1, train_n)))
    ONE = np.ones(train_n - 1)
    B = (1 - gamma) * np.array([-Zy, M2, M3, TAU, ONE]).T

    # 构建 Y
    Y = np.diff(y1)

    # 最小二乘法
    # 最小二乘法
    try:
        a, b2, b3, h1, h2 = np.dot(np.dot(np.linalg.inv(np.dot(B.T, B)), B.T), Y)
    except:
        return 1e9 # 有时会出现几万的mape，如果异常则返回1亿，不会对训练造成冲突

    # 求 mu
    mu1 = (1 - gamma) / (1 + 0.5 * a * (1 - gamma))
    mu2 = (1 - 0.5 * a * (1 - gamma)) / (1 + 0.5 * a * (1 - gamma))
    mu3 = (h1 * (1 - gamma) * (exp(tau) - 1)) / (1 + 0.5 * a * (1 - gamma)) / tau
    mu4 = (h2 * (1 - gamma)) / (1 + 0.5 * a * (1 - gamma))

    # 这里重新获取下原数据，以实现所有数据的预测
    # 第一列是年份，故先舍弃
    X0 = data.iloc[:, 1:].to_numpy()
    X1 = np.array([get_fractional_series(X0[:, 0], zeta),
                   get_fractional_series(X0[:, 1], zeta),
                   get_fractional_series(X0[:, 2], zeta)]).T

    # 求 hat y1(1为上标的那个haty^1)
    hat_y1 = [y1[0]]
    a = []
    for m in range(2, n + 1):
        hat_ym = 0
        hat_ym += mu2 ** (m - 1) * y1[0]
        for v in range(1, m):
            a.append(m - v)
            hat_ym += mu2 ** (v - 1) * mu1 * (b2 * X1[m - v, 1] + b3 * X1[m - v, 2])
        for w in range(0, m - 1):
            hat_ym += mu2 ** w * (mu4 + mu3 * exp(tau * (m - w - 2)))
        hat_y1.append(hat_ym)

    hat_x1 = np.array(hat_y1) ** (1 / (1 - gamma))

    hat_x0 = []
    for k in range(2, n + 1):
        hat_x0_m = 0
        for j in range(0, k):
            hat_x0_m += (-1) ** j * GAMMA(zeta + 1) / (GAMMA(j + 1) * GAMMA(zeta - j + 1)) * hat_x1[k - j - 1]
        hat_x0.append(hat_x0_m)

    original_x1 = data.iloc[1:, 1]
    APE = np.abs(hat_x0 - original_x1) / original_x1
    return  pd.Series([np.mean(APE)] + list(hat_x0[-2:]))

In [4]:
data = pd.read_excel("京津冀3省2014-2023使用版数据-2位小数.xlsx")
data = data.iloc[:, :]
n = data.shape[0]
train_n = n-2

In [5]:
paraments_result = []
paraments = pd.read_excel("case2.xlsx")

In [6]:
a = paraments.apply(lambda x: func(list((x[2], x[3], x[4]))), axis=1)
a['pop'] = paraments[0]
a['max_iter'] = paraments[1]
a.columns = ["MAPE", "倒数第二年预测值", "最后一年预测值", "pop", "max_iter"]

a.to_excel("后俩值.xlsx", index=0)